In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url_train_dev = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTOZ2rC82rhNsJduoyKYTsVeH6ukd7Bpxvxn_afOibn3R-eadZGXu82eCU9IRpl4CK_gefEGsYrA_oM/pub?gid=1863430984&single=true&output=tsv'
url_test = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-KNR9nuYatLkSbzSRgpz6Ku1n4TN4w6kKmFLkA6QJHTfQzmX0puBsLF7PAAQJQAxUpgruDd_RRgK7/pub?gid=417546901&single=true&output=tsv'

# Load and inspect the data

In [3]:
from io import StringIO
import requests

def load_dataset(url):
    r = requests.get(url)
    data = r.content.decode('utf8')
    df = pd.read_csv(StringIO(data), sep='\t')
    df.columns = ['tweet', 'label']
    return df

In [4]:
df_train_dev = load_dataset(url_train_dev)
df_test = load_dataset(url_test)

# Deal with class imbalance by unifying very rare classes (1 member) and upsampling rare classes

In [5]:
df_train_dev['label'] = df_train_dev['label'].apply(lambda x: x.strip().lower())
df_test['label'] = df_test['label'].apply(lambda x: x.strip().lower())

In [6]:
def unify_singleton_classes(df, threshold):
    counts = df['label'].value_counts()
    rare_labels = [label for label in counts[counts < threshold].index]
    df.loc[(df.label.isin(rare_labels)), 'label'] = 'und'
    return df

df_train_dev_uni = unify_singleton_classes(df_train_dev, 3)

In [7]:
from sklearn.utils import resample

def upsample_rare_classes(df, threshold, sample_size):
    """Copy low frequent Tweets of languages, that have a number 
    of tweets below a certain threshold.
    
    :param data_train: dataframe
    :param threshold: int
    :param sample_size: int
    """
    class_counts = defaultdict(int)
    classes_to_upsample = set()
    for index, row in df.iterrows():
        class_counts[row['label']] += 1
    for label, count in class_counts.items():
        if count < threshold:
            classes_to_upsample.add(label)
    df_to_upsample = df[df['label'].isin(classes_to_upsample)]
    upsampled_classes = pd.DataFrame(resample(df_to_upsample.to_numpy(), replace=True, n_samples=sample_size, random_state=42))
    upsampled_classes.columns = ['tweet', 'label']
    return pd.concat([df, upsampled_classes]).sample(frac=1)

df_train_dev_up = upsample_rare_classes(df=df_train_dev_uni, threshold=50, sample_size=2000)

In [8]:
print(df_train_dev.shape)
print(df_train_dev_up.shape)

(52675, 2)
(54675, 2)


# Process labels

In [9]:
from sklearn.preprocessing import LabelEncoder
le_fitted = LabelEncoder().fit(df_train_dev_up['label'])

In [10]:
# map all classes that are not in train_dev to undefined
for i, label in enumerate(df_test['label']):
    df_test['label'][i] = 'und' if label not in le_fitted.classes_ else label
# check if it worked: should return an empty list
print([label for label in df_test['label'] if label not in set(df_train_dev_up['label'])])

[]


In [11]:
y_train_dev, y_test = le_fitted.transform(df_train_dev_up['label']), le_fitted.transform(df_test['label'])

# Preprocess Tweets

Pipeline classes:

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


class TweetNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self 

    def _normalize_tweet(self, tweet):
        """Remove punctuation and newlines, lowercase, pad with spaces.

        :param tweet: string
        :return: normalized string
        """
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))
        tweet = re.sub(r'\n', r'', tweet)
        tweet = tweet.lower()
        tweet = re.sub(r'@\w+\b', r'', tweet)
        tweet = re.sub(r'\b\S+//\S+\b', r'', tweet)
        # tweet = ' ' + tweet + ' '
        return tweet
    
    def transform(self, X, y=None):
        tweets = []
        for tweet in X:
            tweets.append(self._normalize_tweet(tweet))
        return np.array(tweets)


class FeatureExtractor(BaseEstimator, TransformerMixin):


    vowels = set([c for c in 'aeiouäöüàéèëï'])
    consonants = set([c for c in 'bcdfghklmnlpqrstvwxyz'])

    def __init__(self):
        self.scaler = MinMaxScaler()

    def _to_bigrams(self, tweet):
        return [bg[0] + bg[1] for bg in zip(tweet, tweet[1:])]

    def _get_vowel_consonant_ratio(self, tweet):
        vf = 0
        cf = 0
        for c in tweet.lower():
            if c in self.vowels:
                vf =+ 1
            elif c in self.consonants:
                cf += 1
        return vf / (cf + 1)

    def _get_capitalization_ratio(self, tweet):
        up_count = 0
        for c in tweet:
            if c.upper() == c:
                up_count += 1
        return up_count / (len(tweet) + 1)

    def _get_double_char_freq(self, tweet):
        double_freq = 0
        for bg in self._to_bigrams(tweet):
            if bg[0] == bg[1]:
                double_freq += 1
        return double_freq
    
    def _extract_num_features(self, tweets):
        num_features = []
        for tweet in tweets:
            feat_tweet = []
            feat_tweet.append(self._get_vowel_consonant_ratio(tweet))
            feat_tweet.append(self._get_capitalization_ratio(tweet))
            feat_tweet.append(self._get_double_char_freq(tweet))
            num_features.append(feat_tweet)
        return np.array(num_features)
    
    def fit(self, X, y=None):
        numerical_features = self._extract_num_features(X)
        self.scaler.fit(numerical_features)
        return self
    
    def transform(self, X, y=None):
        numerical_features= self._extract_num_features(X)
        return X, self.scaler.transform(numerical_features)


class MatrixToArrayConverter1(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[0].toarray(), X[1]


class MatrixUnifier(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return np.concatenate([X[0].todense(), X[1]], axis=1)


class CountVectorizerWrapper:

    def __init__(self, ngram_range, analyzer, max_features, binary):
        print('args:', str([ngram_range, analyzer, max_features, binary]))
        self.countvec = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer, max_features=max_features, binary=binary)
    
    def fit(self, X, y=None):
        tweets, numerical_features = X
        self.countvec.fit(tweets)
        return self
    
    def transform(self, X, y=None):
        tweets, numerical_features = X
        return self.countvec.transform(tweets), numerical_features


class OneHotEncoderWrapper:

    def __init__(self, handle_unknown):
        self.ohe = OneHotEncoder(handle_unknown=handle_unknown)
    
    def fit(self, X, y=None):
        self.ohe.fit(X[0])
        return self
    
    def transform(self, X, y=None):
        return self.ohe.transform(X[0]), X[1]


Helper classes for the pipeline:

In [13]:
class GenericClassifier(BaseEstimator):

    def __init__(self, estimator):
        self.clf = clf


    def fit(self, X, y=None, **kwargs):
        self.clf.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.clf.predict(X)


    def predict_proba(self, X):
        return self.clf.predict_proba(X)


    def score(self, X, y):
        return self.clf.score(X, y)


class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        import pdb; pdb.set_trace()
        return X

    def fit(self, X, y=None, **fit_params):
        return self

# GridSearch and Training

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [15]:
param_grid = {
    'CLF__hidden_layer_sizes': (50, 100),
    'CLF__max_iter': (20,),
    'CLF__activation': ('logistic', 'relu'),
    'CLF__solver': ('sgd', 'adam'),
    'CLF__early_stopping': (True, False),
}

In [16]:
from sklearn.neural_network import MLPClassifier
bigram_vec_args = dict(ngram_range=(2,2), analyzer='char_wb', max_features=100, binary=True)
pipe = Pipeline(steps=[
    ('TweetNormalizer', TweetNormalizer()),
    ('FeatureExtractor', FeatureExtractor()),
    ('BigramVectorizer', CountVectorizerWrapper(**bigram_vec_args)),
    ('MatrixToArrayConverter', MatrixToArrayConverter1()),
    ('OneHotEncoder', OneHotEncoderWrapper(handle_unknown='ignore')),
    ('MatrixUnifier', MatrixUnifier()),
    ('CLF', MLPClassifier())
], verbose=True)
grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid, scoring='f1_micro', cv=10)
grid.fit(df_train_dev_up['tweet'].to_numpy(), y_train_dev)

args: [(2, 2), 'char_wb', 100, True]
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   1.8s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.2s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   6.8s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.4s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.1s


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[Pipeline] ............... (step 7 of 7) Processing CLF, total=  10.0s
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   1.1s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.1s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   6.8s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.4s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.1s


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[Pipeline] ............... (step 7 of 7) Processing CLF, total=  10.1s
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   1.1s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.1s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   6.7s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.4s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.1s


...

[Pipeline] ............... (step 7 of 7) Processing CLF, total=  14.6s
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   1.1s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.2s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   6.8s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.4s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.0s


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[Pipeline] ............... (step 7 of 7) Processing CLF, total=  14.7s
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   2.3s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.8s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   7.6s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.5s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.0s
[Pipeline] ............... (step 7 of 7) Processing CLF, total=  20.0s


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('TweetNormalizer', TweetNormalizer()),
                                       ('FeatureExtractor', FeatureExtractor()),
                                       ('BigramVectorizer',
                                        <__main__.CountVectorizerWrapper object at 0x7f5ba894d358>),
                                       ('MatrixToArrayConverter',
                                        MatrixToArrayConverter1()),
                                       ('OneHotEncoder',
                                        <__main__.OneHotEncoderWrapper object at 0x7f5ba894d518...
                                                      validation_fraction=0.1,
                                                      verbose=False,
                                                      warm_start=False))],
                                verbose=True),
             iid='deprecated', n_jobs

# Results

Micro f1-Score of the naive base models on the dev set:

In [17]:
grid.cv_results_

{'mean_fit_time': array([23.29655533, 23.80013883, 28.37692676, 28.94297936, 23.9102855 ,
        24.33465533, 29.08181942, 29.76082757, 22.48326707, 23.23607869,
        26.20014956, 27.28121769, 23.15511045, 23.85770602, 27.07712088,
        27.95471535]),
 'mean_score_time': array([0.65281556, 0.64989462, 0.66801858, 0.66540723, 0.6520777 ,
        0.65625441, 0.66749249, 0.67079608, 0.65092449, 0.68989036,
        0.65476646, 0.65378716, 0.64737597, 0.69298468, 0.65165803,
        0.65400846]),
 'mean_test_score': array([0.59623236, 0.74820312, 0.61132147, 0.75776885, 0.60541409,
        0.75067215, 0.61984453, 0.76053056, 0.69968006, 0.76495655,
        0.7096298 , 0.77413811, 0.7055695 , 0.76513954, 0.71347063,
        0.77419294]),
 'param_CLF__activation': masked_array(data=['logistic', 'logistic', 'logistic', 'logistic',
                    'logistic', 'logistic', 'logistic', 'logistic', 'relu',
                    'relu', 'relu', 'relu', 'relu', 'relu', 'relu', 'relu'],
     

F1-micro and f1-macro score of the best mlp model on the test set:

In [21]:
from sklearn.metrics import f1_score
preds = grid.predict(df_test['tweet'].to_numpy())
f1_micro = f1_score(preds, y_test, average='micro')
f1_macro = f1_score(preds, y_test, average='macro')
print(f'F1-micro-score on the testset: {f1_micro}')
print(f'F1-macro-score on the testset: {f1_macro}')

F1-micro-score on the testset: 0.7888395210482717
F1-macro-score on the testset: 0.2545447747657814


Let's check the confusion matrix:

In [22]:
num_classes = len(le_fitted.classes_)
def create_confusion_matrix(num_classes, preds, y_test):
    """Create confusion matrix 'by hand' since test set does not contain all labels (thanks to Sarah Kiener)."""
    df = pd.DataFrame(np.zeros((num_classes, num_classes), dtype=int))
    for i, j in zip(preds, y_test):
        df.iloc[i, j] += 1
    df.columns = le_fitted.classes_
    df.index = le_fitted.classes_
    return df
df = create_confusion_matrix(num_classes, preds, y_test)
df

,ar,ar_latn,bn,bs,ca,cs,da,de,el,en,es,fa,fi,fr,gl,he,hi,hi-latn,hr,hu,id,it,ja,jv,ko,ms,ne,nl,no,pl,pt,ro,ru,sq,sr,su,sv,sw,ta,th,tl,tr,uk,und,ur,ur_latn,vi,zh-cn,zh-tw
ar,153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,65,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,0,0,0,7,1,0,0,0,1
ar_latn,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
bs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ca,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
cs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
da,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
de,0,0,0,0,0,0,0,31,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
el,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
en,3,2,0,0,1,0,1,12,1,4374,53,0,0,45,0,1,0,2,0,0,45,3,19,1,4,3,0,8,1,7,35,1,13,0,0,0,6,0,0,3,13,19,0,214,0,1,0,0,0
